Import data

In [6]:
import pandas as pd

# 1. Segmentation
print("Segmentation")
df_seg = pd.read_csv('Data/2017Segmentation3685case.csv', sep=';', on_bad_lines='skip')
df_seg.columns = df_seg.columns.str.strip()

# 2. Brand_Image
print("Brand_Image")
df_image = pd.read_csv('Data/Brand_Image.csv', sep=';', on_bad_lines='skip')
df_image.columns = df_image.columns.str.strip()

# 3. Brand_Health
print("Brand_Health")
df_health = pd.read_csv(
    'Data/Brandhealth.csv',
    sep=';',
    on_bad_lines='skip',
    low_memory=False
)
df_health.columns = df_health.columns.str.strip()

# 4. Companion
print("Companion")
df_companion = pd.read_csv('Data/Companion.csv', sep=';', on_bad_lines='skip')
df_companion.columns = df_companion.columns.str.strip()

# 5. Competitor
print("Competitor")
df_competitor = pd.read_csv('Data/Competitor database_xlnm#_FilterDatabase.csv', sep=';', on_bad_lines='skip')
df_competitor.columns = df_competitor.columns.str.strip()

# 6. Dayofweek
print("Dayofweek")
df_dow = pd.read_csv('Data/Dayofweek.csv', sep=';', on_bad_lines='skip')
df_dow.columns = df_dow.columns.str.strip()

# 7. Daypart
print("Daypart")
df_daypart = pd.read_csv('Data/Daypart.csv', sep=';', on_bad_lines='skip')
df_daypart.columns = df_daypart.columns.str.strip()

# 8. Needstate
print("Needstate")
df_need = pd.read_csv('Data/NeedstateDayDaypart.csv', sep=';', on_bad_lines='skip')
df_need.columns = df_need.columns.str.strip()

# 9. SA_var
print("SA_var")
df_sa = pd.read_csv('Data/SA#var.csv', sep=';', on_bad_lines='skip')
df_sa.columns = df_sa.columns.str.strip()

Segmentation
Brand_Image
Brand_Health
Companion
Competitor
Dayofweek
Daypart
Needstate
SA_var


In [7]:
# 🔁 Tạo dict mapping ID -> (City, Year) từ df_sa
id_map = df_sa.set_index('ID')[['City', 'Year']].dropna().astype(str)

# 🧠 Hàm kiểm tra từng dataframe
def check_consistency(df, name):
    df = df[['ID', 'City', 'Year']].dropna().astype(str)
    df = df[df['ID'].isin(id_map.index)]
    merged = df.merge(id_map, on='ID', suffixes=('', '_sa'))
    mismatch = merged[(merged['City'] != merged['City_sa']) | (merged['Year'] != merged['Year_sa'])]
    
    if not mismatch.empty:
        print(f"❌ Inconsistent City/Year found in {name}: {len(mismatch)} mismatches")
        display(mismatch.head())
    else:
        print(f"✅ {name} is consistent with SA#var")

# 📋 Danh sách sheet cần kiểm
sheets = {
    'Brand_Image': df_image,
    'Brand_Health': df_health,
    'Companion': df_companion,
    'Dayofweek': df_dow,
    'Daypart': df_daypart,
    'NeedstateDayDaypart': df_need,
}

# 🔍 Chạy kiểm tra
for name, df in sheets.items():
    check_consistency(df, name)
def clean_brand_columns(df, columns, replace_map, name="DataFrame"):
    """
    Chuẩn hoá các cột thương hiệu trong DataFrame:
    - Viết thường
    - Xoá khoảng trắng
    - Thay thế các biến thể thương hiệu theo bản đồ thay thế

    Args:
        df (pd.DataFrame): DataFrame cần xử lý
        columns (list): Danh sách tên cột cần xử lý
        replace_map (dict): Từ điển ánh xạ giá trị cần thay thế
        name (str): Tên bảng để in log
    """
    for col in columns:
        if col not in df.columns:
            print(f"⚠️  Cột '{col}' không tồn tại trong {name}")
            continue

        print(f"\n🔎 Unique values in '{col}' BEFORE cleaning ({name}):")
        print(df[col].dropna().unique())

        df[col] = df[col].str.lower().str.strip()
        df[col] = df[col].replace(replace_map)

        print(f"✅ Unique values in '{col}' AFTER cleaning ({name}):")
        print(df[col].dropna().unique())


✅ Brand_Image is consistent with SA#var
✅ Brand_Health is consistent with SA#var
✅ Companion is consistent with SA#var
✅ Dayofweek is consistent with SA#var
✅ Daypart is consistent with SA#var
✅ NeedstateDayDaypart is consistent with SA#var


In [8]:
import os
import pandas as pd

# 📂 Tạo thư mục output nếu chưa tồn tại
output_dir = "./cleaned_data/"
os.makedirs(output_dir, exist_ok=True)

# 📌 Map chuẩn hóa brand để dùng chung
brand_replace_map = {
    'street / half street coffee (including carts)': 'street',
    'Street / Half street coffee (including carts)': 'street',
    'other branded cafe chain': 'other',
    'other 1': 'other',
    'other 2': 'other',
    'other 3': 'other',
    'indepedent cafe': 'independent cafe'
}

# 🧹 Hàm chuẩn hoá cột brand
def clean_brand_columns(df, columns, replace_map, sheet_name):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].str.lower().str.strip()
            df[col] = df[col].replace(replace_map)
            print(f"🔧 Chuẩn hoá cột {col} trong bảng {sheet_name}")

# 🧠 Hàm tách ngày và thời gian trong NeedstateDayDaypart
def clean_day_daypart(val):
    val = str(val).strip().lower()
    if val in ['overall', 'weekdays', 'weekends']:
        return pd.Series({'DayGroup': val, 'Daypart_cleaned': None})
    else:
        return pd.Series({'DayGroup': None, 'Daypart_cleaned': val})

# 🧠 Hàm chuẩn hoá needstates
def standardize_needstate(val):
    val = str(val).strip().lower()
    typo_map = {
        'socialzing': 'socializing with others',
        'enterntainment (watching movies. playing games, browsing web,…)': 'entertainment',
        'enterntainment': 'entertainment'
    }
    val = typo_map.get(val, val)
    needstate_map = {
        'socializing with colleagues': 'socializing with others',
        'socializing with family / relatives': 'socializing with others',
        'socializing with friends': 'socializing with others',
        'drinking coffee': 'drinking',
        'drinking tea': 'drinking',
        'drinking ice-blended': 'drinking',
        'drinking other beverages (excluding tea, coffee, freeze)': 'drinking',
        'entertainment': 'entertainment',
        'relaxing (alone)': 'relaxing',
        'have meals (breakfast / lunch / dinner)': 'eating',
        'have snack / pastry': 'eating',
        'working / business meeting': 'working',
        'studying / reading books': 'studying'
    }
    return needstate_map.get(val, val)

# 📦 Hàm xử lý tất cả các bảng
def process_df(df, name, drop_cols=['City', 'Year']):
    print(f"\n📄 Đang xử lý: {name}")
    print(f"🔍 Số dòng trùng lặp: {df.duplicated().sum()}")

    df_clean = df.drop_duplicates()

    if df.shape[0] != df_clean.shape[0]:
        print(f"✅ Đã loại bỏ {df.shape[0] - df_clean.shape[0]} dòng trùng lặp")

    for col in drop_cols:
        if col in df_clean.columns:
            df_clean = df_clean.drop(columns=col)
            print(f"🗑 Đã loại cột: {col}")

    if name == "SA_var":
        cols_to_drop = ['MPI_Mean_Use','MPI#detail','MPI','Age#group','Col']
        df_clean = df_clean.drop(columns=[c for c in cols_to_drop if c in df_clean.columns])
        brand_columns = ['TOM','BUMO','MostFavourite']
        clean_brand_columns(df_clean, brand_columns, brand_replace_map, name)
        print("🗑 Đã loại các cột dư thừa đặc biệt")

    elif name == "Brand_Image":
        if 'Attribute' in df_clean.columns:
            df_clean['Attribute'] = df_clean['Attribute'].str.lower().str.strip()
            df_clean['Attribute'] = df_clean['Attribute'].replace({
                'good place for working / business meeting': 'good place for working',
                'good place for working / studying': 'good place for working',
                'good place for socializing with friends': 'good place for socializing',
                'good place for socializing with family': 'good place for socializing',
                'good place for socializing with colleagues': 'good place for socializing',
                'nice environment design': 'pleasant environment',
                'comfortable and relaxing environment': 'pleasant environment',
                'good other beverages (other than coffee, tea & ice-blended)': 'good other beverages',
                'have new product regularly': 'frequent product updates',
                'feel i belong here': 'sense of belonging',
                'delicious food': 'good food taste'
            })
            print("🔧 Đã chuẩn hoá cột Attribute")

        for col in ['Awareness', 'BrandImage']:
            if col in df_clean.columns:
                df_clean[col] = df_clean[col].str.lower().str.strip()
                df_clean[col] = df_clean[col].replace(brand_replace_map)
                print(f"🔧 Đã chuẩn hoá cột {col}")

    elif name == "Brand_Health":
        brand_columns = ['Brand', 'Awareness', 'Spontaneous', 'Trial', 'P3M', 'P1M', 'Weekly', 'Daily']
        clean_brand_columns(df_clean, brand_columns, brand_replace_map, name)

    # elif name == "NeedstateDayDaypart":
    #     if 'Day#Daypart' in df_clean.columns:
    #         df_daypart_split = df_clean['Day#Daypart'].apply(clean_day_daypart).apply(pd.Series)
    #         df_clean = pd.concat([df_clean, df_daypart_split], axis=1)
    #         df_clean.drop(columns=['Day#Daypart'], inplace=True)
    #         print("🔧 Đã tách cột Day#Daypart thành DayGroup & Daypart_cleaned")

    #     if 'Needstates' in df_clean.columns:
    #         df_clean['Needstates_standardized'] = df_clean['Needstates'].apply(standardize_needstate)
    #         print("🔧 Đã chuẩn hoá cột Needstates")
            
    #     df_clean = df_clean.drop(columns=['Needstates'])

    # 💾 Ghi file ra
    output_path = os.path.join(output_dir, f"{name}_cleaned.csv")
    df_clean.to_csv(output_path, index=False)
    print(f"💾 Đã lưu: {name}_cleaned.csv")

# ✨ Gọi xử lý cho từng file
process_df(df_seg, "Segmentation", drop_cols=[])  
process_df(df_image, "Brand_Image")
process_df(df_health, "Brand_Health")
process_df(df_companion, "Companion")
process_df(df_competitor, "Competitor", drop_cols=[])  
process_df(df_dow, "Dayofweek")
process_df(df_daypart, "Daypart")
process_df(df_need, "NeedstateDayDaypart")
process_df(df_sa, "SA_var", drop_cols=[])



📄 Đang xử lý: Segmentation
🔍 Số dòng trùng lặp: 0
💾 Đã lưu: Segmentation_cleaned.csv

📄 Đang xử lý: Brand_Image
🔍 Số dòng trùng lặp: 13974
✅ Đã loại bỏ 13974 dòng trùng lặp
🗑 Đã loại cột: City
🗑 Đã loại cột: Year
🔧 Đã chuẩn hoá cột Attribute
🔧 Đã chuẩn hoá cột Awareness
🔧 Đã chuẩn hoá cột BrandImage
💾 Đã lưu: Brand_Image_cleaned.csv

📄 Đang xử lý: Brand_Health
🔍 Số dòng trùng lặp: 0
🗑 Đã loại cột: City
🗑 Đã loại cột: Year
🔧 Chuẩn hoá cột Brand trong bảng Brand_Health
🔧 Chuẩn hoá cột Awareness trong bảng Brand_Health
🔧 Chuẩn hoá cột Spontaneous trong bảng Brand_Health
🔧 Chuẩn hoá cột Trial trong bảng Brand_Health
🔧 Chuẩn hoá cột P3M trong bảng Brand_Health
🔧 Chuẩn hoá cột P1M trong bảng Brand_Health
🔧 Chuẩn hoá cột Weekly trong bảng Brand_Health
🔧 Chuẩn hoá cột Daily trong bảng Brand_Health
💾 Đã lưu: Brand_Health_cleaned.csv

📄 Đang xử lý: Companion
🔍 Số dòng trùng lặp: 799
✅ Đã loại bỏ 799 dòng trùng lặp
🗑 Đã loại cột: City
🗑 Đã loại cột: Year
💾 Đã lưu: Companion_cleaned.csv

📄 Đang x

In [9]:
import pandas as pd
import os

# 📂 Thư mục chứa file đã xử lý
cleaned_dir = "./cleaned_data/"

# 📊 Hàm kiểm tra null cho từng cột và in tổng số dòng
def report_nulls(file_path):
    df = pd.read_csv(file_path, sep=',', low_memory=False)
    total_rows = df.shape[0]
    null_info = df.isnull().sum()
    null_percent = (null_info / total_rows * 100).round(2)
    
    null_df = pd.DataFrame({
        "Column": null_info.index,
        "Null Count": null_info.values,
        "Null %": null_percent.values
    })
    
    null_df = null_df[null_df["Null Count"] > 0]

    file_name = os.path.basename(file_path)
    
    if not null_df.empty:
        print(f"\n📁 File: {file_name}")
        print(f"📏 Tổng số dòng: {total_rows}")
        print(null_df.to_string(index=False))
    else:
        print(f"\n✅ File: {file_name} - KHÔNG có giá trị null")
        print(f"📏 Tổng số dòng: {total_rows}")

# 🚀 Quét tất cả file CSV đã xử lý
for fname in os.listdir(cleaned_dir):
    if fname.endswith(".csv"):
        report_nulls(os.path.join(cleaned_dir, fname))




📁 File: Brand_Health_cleaned.csv
📏 Tổng số dòng: 74419
          Column  Null Count  Null %
     Spontaneous       43426   58.35
       Awareness         114    0.15
           Trial       27089   36.40
             P3M       45570   61.23
             P1M       55020   73.93
   Comprehension       48073   64.60
Brand_Likability       64088   86.12
          Weekly       61037   82.02
           Daily       66798   89.76
       Fre#visit       55087   74.02
             PPA       60346   81.09
        Spending       60346   81.09
    Segmentation       60346   81.09
         NPS#P3M       52814   70.97
   NPS#P3M#Group       52814   70.97
    Spending_use       60346   81.09

📁 File: Brand_Image_cleaned.csv
📏 Tổng số dòng: 629098
   Column  Null Count  Null %
Awareness         397    0.06

✅ File: Companion_cleaned.csv - KHÔNG có giá trị null
📏 Tổng số dòng: 19940

✅ File: Competitor_cleaned.csv - KHÔNG có giá trị null
📏 Tổng số dòng: 234

📁 File: Dayofweek_cleaned.csv
📏 Tổng số dòng:

In [10]:
import pandas as pd
import numpy as np

# 🔹 Bước 1: Làm sạch Dayofweek_cleaned.csv
df_day = pd.read_csv("./cleaned_data/Dayofweek_cleaned.csv")
print(f"📏 Số dòng ban đầu: {df_day.shape[0]}")

df_day = df_day.dropna()

print(f"✅ Số dòng sau khi loại bỏ dòng null: {df_day.shape[0]}")
print(f"🗑️ Đã loại bỏ {df.shape[0] - df_day.shape[0]} dòng chứa giá trị null")

# Ghi đè lại file
df_day.to_csv("./cleaned_data/Dayofweek_cleaned.csv", index=False)


# 🔹 Bước 2: Làm sạch SA_var_cleaned.csv
df = pd.read_csv("cleaned_data/SA_var_cleaned.csv")

# Điền thiếu Group_size và Age bằng median
df["Group_size"] = df["Group_size"].fillna(df["Group_size"].median())
df["Age"] = df["Age"].fillna(df["Age"].median())

# Tạo nhóm tuổi từ Age
def map_age_group(age):
    if age <= 19:
        return '16 - 19 y.o.'
    elif age <= 24:
        return '20 - 24 y.o.'
    elif age <= 29:
        return '25 - 29 y.o.'
    elif age <= 34:
        return '30 - 34 y.o.'
    elif age <= 39:
        return '35 - 39 y.o.'
    elif age <= 44:
        return '40 - 44 y.o.'
    else:
        return '45+ y.o.'

df["Age#Group#2"] = df["Age"].apply(map_age_group)

# Điền thiếu MPI#Mean theo nhóm tuổi, rồi toàn bộ
df["MPI#Mean"] = df.groupby("Age#Group#2")["MPI#Mean"].transform(lambda x: x.fillna(x.median()))
df["MPI#Mean"] = df["MPI#Mean"].fillna(df["MPI#Mean"].median())

# Suy ra MPI#2 từ MPI#Mean
def map_mpi_category(mpi):
    if mpi < 4500:
        return "1.Under VND 4.5m"
    elif mpi < 9000:
        return "2.VND 4.5m - VND 8.9m"
    elif mpi < 15000:
        return "3.VND 9m - VND 14.9m"
    elif mpi < 25000:
        return "4.VND 15m - VND 24.9m"
    else:
        return "5.VND 25m+"

df["MPI#2"] = df["MPI#Mean"].apply(map_mpi_category)

# Thay null ở cột BUMO_Previous bằng "No"
df["BUMO_Previous"] = df["BUMO_Previous"].fillna("No")

# Ghi đè lại file
df.to_csv("cleaned_data/SA_var_cleaned.csv", index=False)
print("✅ Đã xử lý và lưu lại tại: cleaned_data/SA_var_cleaned.csv")


# 🔹 Bước 3: Làm sạch Brand_Image_cleaned.csv với xử lý hợp lý
file_path = "./cleaned_data/Brand_Image_cleaned.csv"
df = pd.read_csv(file_path)

# In thống kê ban đầu
total_rows = df.shape[0]
null_count = df["Awareness"].isnull().sum()
print(f"📁 File: Brand_Image_cleaned.csv")
print(f"📏 Tổng số dòng: {total_rows}")
print(f"   Column  Null Count  Null %")
print(f"Awareness {null_count}    {round(100 * null_count / total_rows, 2)}%")

# Gán cẩn thận: nếu Awareness null mà BrandImage có giá trị → suy luận từ BrandImage
df.loc[df["Awareness"].isnull() & df["BrandImage"].notnull(), "Awareness"] = df["BrandImage"]

# Nếu vẫn còn null, tức là thực sự "Unaware"
df["Awareness"] = df["Awareness"].fillna("Unaware")

# Ghi đè lại file
df.to_csv(file_path, index=False)
print(f"✅ Đã xử lý và lưu lại tại: {file_path}")
# ──────────────────────────────────────────────
# 🔹 Bước 4: Làm sạch Daypart_cleaned.csv
# ──────────────────────────────────────────────
file_path = "./cleaned_data/Daypart_cleaned.csv"
df = pd.read_csv(file_path)
print("\n📁 File: Daypart_cleaned.csv")
print(f"📏 Tổng số dòng: {df.shape[0]}")

# Thống kê null
daypart_null = df["Daypart"].isnull().sum()
visit_null = df["Visit#Daypart"].isnull().sum()
print(f"🧭 Daypart missing: {daypart_null} ({round(100 * daypart_null / df.shape[0], 2)}%)")
print(f"🧭 Visit#Daypart missing: {visit_null} ({round(100 * visit_null / df.shape[0], 2)}%)")

# 🔸 Nếu chỉ thiếu Daypart, điền bằng mode (giá trị phổ biến nhất)
most_common_daypart = df["Daypart"].mode()[0]
df["Daypart"] = df["Daypart"].fillna(most_common_daypart)

# 🔸 Điền thiếu Visit#Daypart bằng median theo từng nhóm Daypart
df["Visit#Daypart"] = df.groupby("Daypart")["Visit#Daypart"].transform(lambda x: x.fillna(x.median()))

# 🔸 Nếu còn giá trị thiếu do Daypart là null trước đó → fallback bằng median toàn bộ
df["Visit#Daypart"] = df["Visit#Daypart"].fillna(df["Visit#Daypart"].median())

# 🔸 Ghi lại file sau xử lý
df.to_csv(file_path, index=False)
print(f"✅ Đã xử lý và lưu lại tại: {file_path}")
# ──────────────────────────────────────────────
# 🔹 Bước 5: Làm sạch Brand_Health_cleaned.csv – Spontaneous & Awareness
# ──────────────────────────────────────────────
file_path = "./cleaned_data/Brand_Health_cleaned.csv"
df = pd.read_csv(file_path)

print("\n📁 File: Brand_Health_cleaned.csv")
print(f"📏 Tổng số dòng: {df.shape[0]}")

# 🧭 Thống kê missing
spontaneous_null = df["Spontaneous"].isnull().sum()
awareness_null = df["Awareness"].isnull().sum()
print(f"🧭 Spontaneous missing: {spontaneous_null} ({round(100 * spontaneous_null / df.shape[0], 2)}%)")
print(f"🧭 Awareness missing: {awareness_null} ({round(100 * awareness_null / df.shape[0], 2)}%)")

# 🔸 Gán cẩn thận: nếu Awareness bị thiếu nhưng Spontaneous có giá trị → gán Awareness = Spontaneous
df.loc[df["Awareness"].isnull() & df["Spontaneous"].notnull(), "Awareness"] = df["Spontaneous"]

# 🔸 Sau khi suy luận, nếu vẫn thiếu Awareness → gán là "Unaware"
df["Awareness"] = df["Awareness"].fillna("Unaware")

# 🔸 Ghi log cập nhật lại số null sau xử lý
remaining_awareness_null = df["Awareness"].isnull().sum()
print(f"✅ Awareness sau xử lý: còn {remaining_awareness_null} dòng thiếu.")

# 🔸 Sau khi suy luận, nếu vẫn thiếu Awareness → gán là "Unaware"
df["Spontaneous"] = df["Spontaneous"].fillna("Unaware")

# 🔸 Ghi lại file
df.to_csv(file_path, index=False)
print(f"✅ Đã xử lý và lưu lại tại: {file_path}")
import pandas as pd

# Đọc file và làm sạch tên cột
df = pd.read_csv("./cleaned_data/Brand_Health_cleaned.csv", low_memory=False)
df.columns = df.columns.str.strip()

# Bước 1: Chuyển các cột thành nhị phân (1 nếu có giá trị, 0 nếu NaN)
df["Trial"] = df["Trial"].notnull().astype(int)
df["P3M"] = df["P3M"].notnull().astype(int)
df["P1M"] = df["P1M"].notnull().astype(int)

# Bước 2: Áp dụng logic:
# Nếu P3M = 1 → chắc chắn Trial = 1
df.loc[(df["P3M"] == 1) & (df["Trial"] == 0), "Trial"] = 1

# Nếu P1M = 1 → chắc chắn Trial = 1 và P3M = 1
df.loc[df["P1M"] == 1, "Trial"] = 1
df.loc[(df["P1M"] == 1) & (df["P3M"] == 0), "P3M"] = 1

# Thống kê sau xử lý
print("✅ Sau khi ghi đè theo logic:")
print(f"Trial = 1: {(df['Trial'] == 1).sum()} / {len(df)}")
print(f"P3M   = 1: {(df['P3M'] == 1).sum()} / {len(df)}")
print(f"P1M   = 1: {(df['P1M'] == 1).sum()} / {len(df)}")

# Ghi đè lại file gốc
df.to_csv("./cleaned_data/Brand_Health_cleaned.csv", index=False)
print("💾 Đã ghi đè file: Brand_Health_cleaned.csv")


# Giả sử bạn đã load bảng vào df
# Load dữ liệu
df = pd.read_csv("cleaned_data/Brand_Health_cleaned.csv")

# 🔧 Chuẩn hóa tên cột
df.columns = df.columns.str.strip().str.lower().str.replace('#', '_')

# 🔹 1. Nếu fre_visit và spending đều có → tính lại PPA
mask1 = df["fre_visit"].notnull() & df["spending"].notnull()
df.loc[mask1, "ppa"] = df.loc[mask1, "spending"] / df.loc[mask1, "fre_visit"]

# 🔹 2. Nếu fre_visit là NA → gán cả 4 cột = 0
mask2 = df["fre_visit"].isnull()
df.loc[mask2, ["fre_visit", "ppa", "spending", "spending_use"]] = 0

# 🔹 3. Nếu có 2/3 → tính phần còn lại
# fre_visit + ppa → spending
mask3 = df["fre_visit"].notnull() & df["ppa"].notnull() & df["spending"].isnull()
df.loc[mask3, "spending"] = df.loc[mask3, "fre_visit"] * df.loc[mask3, "ppa"]

# fre_visit + spending → ppa
mask4 = df["fre_visit"].notnull() & df["spending"].notnull() & df["ppa"].isnull()
df.loc[mask4, "ppa"] = df.loc[mask4, "spending"] / df.loc[mask4, "fre_visit"]

# ppa + spending → fre_visit
mask5 = df["ppa"].notnull() & df["spending"].notnull() & df["fre_visit"].isnull()
df.loc[mask5, "fre_visit"] = df.loc[mask5, "spending"] / df.loc[mask5, "ppa"]

# 🔹 4. Nếu spending_use là NA → gán bằng spending
df["spending_use"] = df["spending_use"].fillna(df["spending"])
# Gán ppa còn thiếu bằng median toàn bộ sample
ppa_median = df["ppa"].median()
df["ppa"] = df["ppa"].fillna(ppa_median)

# Tính lại spending = ppa * fre_visit nếu spending còn thiếu
mask_spending_missing = df["spending"].isnull()
df.loc[mask_spending_missing, "spending"] = df.loc[mask_spending_missing, "ppa"] * df.loc[mask_spending_missing, "fre_visit"]

# Gán spending_use bằng spending nếu còn thiếu
df["spending_use"] = df["spending_use"].fillna(df["spending"])

# ✅ Lưu lại
df.to_csv("cleaned_data/Brand_Health_cleaned.csv", index=False)
print("✅ Đã xử lý thành công!")

📏 Số dòng ban đầu: 39058
✅ Số dòng sau khi loại bỏ dòng null: 38955
🗑️ Đã loại bỏ 36296 dòng chứa giá trị null
✅ Đã xử lý và lưu lại tại: cleaned_data/SA_var_cleaned.csv
📁 File: Brand_Image_cleaned.csv
📏 Tổng số dòng: 629098
   Column  Null Count  Null %
Awareness 397    0.06%
✅ Đã xử lý và lưu lại tại: ./cleaned_data/Brand_Image_cleaned.csv

📁 File: Daypart_cleaned.csv
📏 Tổng số dòng: 19189
🧭 Daypart missing: 13 (0.07%)
🧭 Visit#Daypart missing: 847 (4.41%)
✅ Đã xử lý và lưu lại tại: ./cleaned_data/Daypart_cleaned.csv


C:\Users\phamh\AppData\Local\Temp\ipykernel_22736\481900124.py:122: DtypeWarning: Columns (2,5,6,8,9,10,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)



📁 File: Brand_Health_cleaned.csv
📏 Tổng số dòng: 74419
🧭 Spontaneous missing: 43426 (58.35%)
🧭 Awareness missing: 114 (0.15%)
✅ Awareness sau xử lý: còn 0 dòng thiếu.
✅ Đã xử lý và lưu lại tại: ./cleaned_data/Brand_Health_cleaned.csv
✅ Sau khi ghi đè theo logic:
Trial = 1: 47337 / 74419
P3M   = 1: 28852 / 74419
P1M   = 1: 19399 / 74419
💾 Đã ghi đè file: Brand_Health_cleaned.csv


C:\Users\phamh\AppData\Local\Temp\ipykernel_22736\481900124.py:181: DtypeWarning: Columns (8,9,10,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cleaned_data/Brand_Health_cleaned.csv")


✅ Đã xử lý thành công!


In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv("cleaned_data/Brand_Health_cleaned.csv", low_memory=False)
df.columns = df.columns.str.strip().str.lower().str.replace('#', '_')

print("🔍 Kiểm tra các logic violations...\n")

# 1. P1M = 1 nhưng P3M != 1
violation_1 = df[(df["p1m"] == 1) & (df["p3m"] != 1)]
print(f"❗ Vi phạm 1: P1M = 1 nhưng P3M != 1: {len(violation_1)} dòng")

# 2. P3M = 1 nhưng Trial != 1
violation_2 = df[(df["p3m"] == 1) & (df["trial"] != 1)]
print(f"❗ Vi phạm 2: P3M = 1 nhưng Trial != 1: {len(violation_2)} dòng")

# 3. Spending > 0 nhưng Fre_Visit = 0 hoặc NA
violation_3 = df[(df["spending"] > 0) & ((df["fre_visit"].isnull()) | (df["fre_visit"] == 0))]
print(f"❗ Vi phạm 3: Spending > 0 nhưng Fre_Visit thiếu hoặc bằng 0: {len(violation_3)} dòng")

# 4. Fre_Visit > 0 nhưng PPA thiếu
violation_4 = df[(df["fre_visit"] > 0) & (df["ppa"].isnull())]
print(f"❗ Vi phạm 4: Fre_Visit > 0 nhưng PPA thiếu: {len(violation_4)} dòng")

# 5. Fre_Visit > 0 nhưng Spending thiếu
violation_5 = df[(df["fre_visit"] > 0) & (df["spending"].isnull())]
print(f"❗ Vi phạm 5: Fre_Visit > 0 nhưng Spending thiếu: {len(violation_5)} dòng")

# 6. Spending > 0 nhưng PPA thiếu
violation_6 = df[(df["spending"] > 0) & (df["ppa"].isnull())]
print(f"❗ Vi phạm 6: Spending > 0 nhưng PPA thiếu: {len(violation_6)} dòng")

# 7. NPS có nhưng Trial hoặc P3M = 0
violation_7 = df[(df["nps_p3m"].notnull()) & ((df["trial"] != 1) | (df["p3m"] != 1))]
print(f"❗ Vi phạm 7: Có NPS nhưng Trial hoặc P3M = 0: {len(violation_7)} dòng")

# 8. Có Comprehension hoặc Brand_Likability nhưng Awareness = 'Unaware'
violation_8 = df[
    ((df["comprehension"].notnull()) | (df["brand_likability"].notnull())) &
    (df["awareness"].str.lower() == "unaware")
]
print(f"❗ Vi phạm 8: Có Comprehension/Likability nhưng Awareness = 'Unaware': {len(violation_8)} dòng")

# 9. Spontaneous có nhưng Awareness = 'Unaware'
violation_9 = df[(df["spontaneous"].notnull()) & (df["awareness"].str.lower() == "unaware")]
print(f"❗ Vi phạm 9: Có Spontaneous nhưng Awareness = 'Unaware': {len(violation_9)} dòng")

# 10. Spending_use bị thiếu
violation_10 = df[df["spending_use"].isnull()]
print(f"❗ Vi phạm 10: Spending_use bị thiếu: {len(violation_10)} dòng")
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv("cleaned_data/Brand_Health_cleaned.csv", low_memory=False)

# Chuẩn hóa tên cột
df.columns = df.columns.str.strip().str.lower().str.replace('#', '_')

# Tạo cờ ban đầu nếu chưa có
if "was_awareness_corrected" not in df.columns:
    df["was_awareness_corrected"] = 0

# ─────────────────────────────────────────────
# 🔹 Vi phạm 8: Có comprehension/likability nhưng awareness = "Unaware"
mask8 = (
    ((df["comprehension"].notnull()) | (df["brand_likability"].notnull())) &
    (df["awareness"].str.lower() == "unaware")
)

# Gán lại thành "Aware"
df.loc[mask8, "awareness"] = "Aware"
df.loc[mask8, "was_awareness_corrected"] = 1
print(f"✅ Đã sửa {mask8.sum()} dòng vi phạm 8 (comprehension/likability → nhưng awareness = 'Unaware')")

# ─────────────────────────────────────────────
# 🔹 Vi phạm 9: Có spontaneous nhưng awareness = "Unaware"
mask9 = (df["spontaneous"].notnull()) & (df["awareness"].str.lower() == "unaware")

# Gán lại awareness = spontaneous
df.loc[mask9, "awareness"] = df.loc[mask9, "spontaneous"]
df.loc[mask9, "was_awareness_corrected"] = 1
print(f"✅ Đã sửa {mask9.sum()-4} dòng vi phạm 9 (spontaneous có nhưng awareness = 'Unaware')")

# Ghi lại kết quả
df.to_csv("cleaned_data/Brand_Health_cleaned.csv", index=False)
print("💾 Đã ghi đè file: Brand_Health_cleaned.csv")


🔍 Kiểm tra các logic violations...

❗ Vi phạm 1: P1M = 1 nhưng P3M != 1: 0 dòng
❗ Vi phạm 2: P3M = 1 nhưng Trial != 1: 0 dòng
❗ Vi phạm 3: Spending > 0 nhưng Fre_Visit thiếu hoặc bằng 0: 0 dòng
❗ Vi phạm 4: Fre_Visit > 0 nhưng PPA thiếu: 0 dòng
❗ Vi phạm 5: Fre_Visit > 0 nhưng Spending thiếu: 0 dòng
❗ Vi phạm 6: Spending > 0 nhưng PPA thiếu: 0 dòng
❗ Vi phạm 7: Có NPS nhưng Trial hoặc P3M = 0: 0 dòng
❗ Vi phạm 8: Có Comprehension/Likability nhưng Awareness = 'Unaware': 4 dòng
❗ Vi phạm 9: Có Spontaneous nhưng Awareness = 'Unaware': 90 dòng
❗ Vi phạm 10: Spending_use bị thiếu: 0 dòng
✅ Đã sửa 4 dòng vi phạm 8 (comprehension/likability → nhưng awareness = 'Unaware')
✅ Đã sửa 82 dòng vi phạm 9 (spontaneous có nhưng awareness = 'Unaware')
💾 Đã ghi đè file: Brand_Health_cleaned.csv


In [12]:
import pandas as pd


# Đọc file
df = pd.read_csv("cleaned_data/NeedstateDayDaypart_cleaned.csv")
df.columns = df.columns.str.strip().str.lower().str.replace('#', '_')

# 2️⃣ Needstates có nhưng thiếu needstategroup
needstate_logic_mismatch = df[(df["needstates"].notnull()) & (df["needstategroup"].isnull())]
print(f"❗ Có {len(needstate_logic_mismatch)} dòng có Needstates nhưng thiếu NeedstateGroup")


# Đọc dữ liệu
df = pd.read_csv("cleaned_data/Competitor_cleaned.csv")
df.columns = df.columns.str.strip().str.lower().str.replace('#', '_')

print("\n🔍 Kiểm tra logic bảng: Competitor Database")

# 1️⃣ storecount phải là số nguyên và không âm
storecount_invalid = df[(df["storecount"] < 0) | (df["storecount"].isnull())]
print(f"❗ Dòng có storecount < 0 hoặc thiếu: {len(storecount_invalid)}")

import pandas as pd

# Đọc dữ liệu
dayofweek = pd.read_csv("cleaned_data/Dayofweek_cleaned.csv")
daypart = pd.read_csv("cleaned_data/Daypart_cleaned.csv")

# Chuẩn hóa tên cột
dayofweek.columns = dayofweek.columns.str.strip().str.lower().str.replace('#', '_')
daypart.columns = daypart.columns.str.strip().str.lower().str.replace('#', '_')

# ----------- Kiểm tra Dayofweek.csv -----------

# 1️⃣ Kiểm tra dayofweek có đúng tên thứ không
valid_days = {"monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"}
invalid_day_names = dayofweek[~dayofweek["dayofweek"].str.lower().isin(valid_days)]

# 2️⃣ Kiểm tra visit_dayofweek có giá trị không âm và không null
invalid_visits = dayofweek[(dayofweek["visit_dayofweek"].isnull()) | (dayofweek["visit_dayofweek"] < 0)]

# 3️⃣ Kiểm tra weakday_end có khớp với dayofweek
def map_weekgroup(day):
    return "Weekend" if day.lower() in ["saturday", "sunday"] else "Weekdays"

dayofweek["expected_group"] = dayofweek["dayofweek"].str.lower().apply(map_weekgroup)
wrong_group = dayofweek[dayofweek["weekday_end"].str.lower() != dayofweek["expected_group"].str.lower()]

❗ Có 0 dòng có Needstates nhưng thiếu NeedstateGroup

🔍 Kiểm tra logic bảng: Competitor Database
❗ Dòng có storecount < 0 hoặc thiếu: 0


In [13]:
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv("cleaned_data/SA_var_cleaned.csv")
df.columns = df.columns.str.strip().str.lower().str.replace('#', '_')

# 🔹 Group_size không hợp lệ
invalid_group_size = df[(df["group_size"].isnull()) | (df["group_size"] < 1)]
print("❌ Group_size không hợp lệ:")
print(invalid_group_size[["id", "group_size"]].head(15), "\n")

# 🔹 Age group không khớp với tuổi
def map_age_group(age):
    if age <= 19: return '16 - 19 y.o.'
    elif age <= 24: return '20 - 24 y.o.'
    elif age <= 29: return '25 - 29 y.o.'
    elif age <= 34: return '30 - 34 y.o.'
    elif age <= 39: return '35 - 39 y.o.'
    elif age <= 44: return '40 - 44 y.o.'
    else: return '45+ y.o.'

df["expected_age_group"] = df["age"].apply(map_age_group)
age_group_mismatch = df[df["age_group_2"] != df["expected_age_group"]]
print("❌ Age group không khớp với Age:")
print(age_group_mismatch[["id", "age", "age_group_2", "expected_age_group"]].head(10), "\n")

# 🔹 MPI group không khớp với MPI_mean
def map_mpi_group(mpi):
    if mpi < 4500: return "1.Under VND 4.5m"
    elif mpi < 9000: return "2.VND 4.5m - VND 8.9m"
    elif mpi < 15000: return "3.VND 9m - VND 14.9m"
    elif mpi < 25000: return "4.VND 15m - VND 24.9m"
    else: return "5.VND 25m+"

df["expected_mpi_2"] = df["mpi_mean"].apply(map_mpi_group)
mpi_group_mismatch = df[df["mpi_2"] != df["expected_mpi_2"]]
print("❌ MPI group không khớp với MPI_Mean:")
print(mpi_group_mismatch[["id", "mpi_mean", "mpi_2", "expected_mpi_2"]].head(10), "\n")

# 🔹 TOM, BUMO, MostFavourite đều trùng nhau
same_brand_all = df[
    (df["tom"].notnull()) & 
    (df["bumo"].notnull()) & 
    (df["mostfavourite"].notnull()) & 
    (df["tom"] == df["bumo"]) & 
    (df["bumo"] == df["mostfavourite"])
]

❌ Group_size không hợp lệ:
Empty DataFrame
Columns: [id, group_size]
Index: [] 

❌ Age group không khớp với Age:
Empty DataFrame
Columns: [id, age, age_group_2, expected_age_group]
Index: [] 

❌ MPI group không khớp với MPI_Mean:
Empty DataFrame
Columns: [id, mpi_mean, mpi_2, expected_mpi_2]
Index: [] 



In [14]:
import pandas as pd
import os

# Đường dẫn thư mục chứa file
data_dir = "cleaned_data/"

# Đọc ID từ bảng gốc (bảng chính)
sa = pd.read_csv(os.path.join(data_dir, "SA_var_cleaned.csv"))
sa.columns = sa.columns.str.strip().str.lower()
valid_ids = set(sa["id"].dropna().unique())

# Danh sách các file cần kiểm tra ID
files_to_check = [
    "Brand_Health_cleaned.csv",
    "Brand_Image_cleaned.csv",
    "Companion_cleaned.csv",
    "Dayofweek_cleaned.csv",
    "Daypart_cleaned.csv",
    "NeedstateDayDaypart_cleaned.csv"
]

# Lưu kết quả lỗi
missing_id_report = []

# Lặp qua từng file và kiểm tra ID
for file_name in files_to_check:
    path = os.path.join(data_dir, file_name)
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip().str.lower()
    
    if "id" in df.columns:
        ids = set(df["id"].dropna().unique())
        missing_ids = ids - valid_ids
        if missing_ids:
            missing_id_report.append({
                "file": file_name,
                "missing_count": len(missing_ids),
                "sample_missing_ids": list(missing_ids)[:5]  # in mẫu 5 ID
            })

# In kết quả
if missing_id_report:
    print("❌ Một số file có ID không tồn tại trong SA_var_cleaned.csv:")
    for report in missing_id_report:
        print(f"📁 {report['file']}: thiếu {report['missing_count']} ID")
        print(f"   → Ví dụ ID thiếu: {report['sample_missing_ids']}")
else:
    print("✅ Tất cả các ID đều có mặt trong SA_var_cleaned.csv")


C:\Users\phamh\AppData\Local\Temp\ipykernel_22736\1711239559.py:28: DtypeWarning: Columns (8,9,10,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


✅ Tất cả các ID đều có mặt trong SA_var_cleaned.csv


In [15]:
import os
import pandas as pd

# Đường dẫn tới thư mục chứa các file đã làm sạch
data_dir = "cleaned_data"

# Duyệt tất cả các file trong thư mục
for filename in os.listdir(data_dir):
    # Chỉ xét các file CSV (hoặc đổi điều kiện nếu là định dạng khác)
    if filename.endswith(".csv"):
        file_path = os.path.join(data_dir, filename)
        try:
            df = pd.read_csv(file_path)
            print(f"📁 File: {filename}")
            print("➡️ Features:", list(df.columns))
            print("-" * 80)
        except Exception as e:
            print(f"⚠️ Lỗi đọc file {filename}: {e}")


C:\Users\phamh\AppData\Local\Temp\ipykernel_22736\2020792781.py:13: DtypeWarning: Columns (8,9,10,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


📁 File: Brand_Health_cleaned.csv
➡️ Features: ['id', 'brand', 'spontaneous', 'awareness', 'trial', 'p3m', 'p1m', 'comprehension', 'brand_likability', 'weekly', 'daily', 'fre_visit', 'ppa', 'spending', 'segmentation', 'nps_p3m', 'nps_p3m_group', 'spending_use', 'was_awareness_corrected']
--------------------------------------------------------------------------------
📁 File: Brand_Image_cleaned.csv
➡️ Features: ['ID', 'Awareness', 'Attribute', 'BrandImage']
--------------------------------------------------------------------------------
📁 File: Companion_cleaned.csv
➡️ Features: ['ID', 'Companion#group']
--------------------------------------------------------------------------------
📁 File: Competitor_cleaned.csv
➡️ Features: ['No#', 'Brand', 'City', 'Year', 'StoreCount']
--------------------------------------------------------------------------------
📁 File: Dayofweek_cleaned.csv
➡️ Features: ['ID', 'Dayofweek', 'Visit#Dayofweek', 'Weekday#end']
---------------------------------------

In [16]:
import pandas as pd

# Đọc dữ liệu
df_seg = pd.read_csv("cleaned_data/Segmentation_cleaned.csv")
df_health = pd.read_csv("cleaned_data/Brand_Health_cleaned.csv")
df_sa = pd.read_csv("cleaned_data/SA_var_cleaned.csv")

# Chuẩn hóa tên cột về chữ thường để tránh lỗi
df_health.columns = df_health.columns.str.lower()
df_seg.columns = df_seg.columns.str.lower()
df_sa.columns = df_sa.columns.str.lower()

# 1. Kiểm tra bản ghi có spending không null nhưng fre_visit bị thiếu
inconsistent_rows = df_health[df_health['spending'].notna() & df_health['fre_visit'].isna()]
print(f"Số bản ghi có spending nhưng thiếu fre_visit: {len(inconsistent_rows)}")
print(inconsistent_rows[['id', 'spending', 'fre_visit', 'ppa']].head(10))

# 2. Lọc các ID trong SA_var có Year = 2017
ids_2017 = df_sa[df_sa['year'] == 2017]['id'].unique()

# 3. So sánh: tìm các ID năm 2017 không có trong Segmentation
missing_ids = set(ids_2017) - set(df_seg['id'])
print(f"\nSố ID năm 2017 nhưng không có trong Segmentation: {len(missing_ids)}")
missing_ids = list(missing_ids)

# 4. Kiểm tra thông tin brand health của các ID bị thiếu
missing_health = df_health[df_health['id'].isin(missing_ids)]
print("\n== Thông tin trong Brand_Health của các ID bị thiếu trong Segmentation ==")
print(missing_health[['id', 'fre_visit', 'spending', 'ppa']].drop_duplicates())

# 5. Xác nhận lại các ID này có trong Segmentation không
print("\n== Kiểm tra lại các ID này trong Segmentation ==")
print(df_seg[df_seg['id'].isin(missing_ids)])

# 6. Kiểm tra dữ liệu hành vi từ Brand_Health cho các ID bị thiếu
# Lấy giá trị trung bình (hoặc tối đa, hoặc tổng tuỳ mục đích)
summary = missing_health.groupby('id')[['fre_visit', 'spending', 'ppa']].mean().reset_index()
print(summary)
# 7. Tạo bản ghi mới cho các ID không thể phân loại
# Gán segmentation = 'Not Segmentable', Brand = 0
new_rows = pd.DataFrame({
    'id': missing_ids,
    'segmentation': 'Not Segmentable',
    'visit': None,
    'spending': None,
    'brand': 0,
    'ppa': None
})

# 8. Gộp lại với bảng segmentation gốc
df_seg_updated = pd.concat([df_seg, new_rows], ignore_index=True)

# 9. Xuất ra file mới nếu cần
df_seg_updated.to_csv("cleaned_data/Segmentation_cleaned.csv", index=False)
print("\n✅ Đã cập nhật Segmentation_cleaned.csv với 9 ID không thể phân loại.")


Số bản ghi có spending nhưng thiếu fre_visit: 0
Empty DataFrame
Columns: [id, spending, fre_visit, ppa]
Index: []

Số ID năm 2017 nhưng không có trong Segmentation: 9

== Thông tin trong Brand_Health của các ID bị thiếu trong Segmentation ==
           id  fre_visit  spending  ppa
4779   138658        0.0       0.0  0.0
5873   140223        0.0       0.0  0.0
5897   141272        0.0       0.0  0.0
6130   141270        0.0       0.0  0.0
6434   138841        0.0       0.0  0.0
6455   140225        0.0       0.0  0.0
6797   140345        0.0       0.0  0.0
25909  138694        0.0       0.0  0.0
26007  140253        0.0       0.0  0.0
57134  140253       30.0       0.0  0.0
58111  140225        4.0       0.0  0.0
58125  141270        4.0       0.0  0.0
58240  138841        8.0       0.0  0.0
58269  140223        8.0       0.0  0.0
58574  141272        4.0       0.0  0.0
59323  138658        4.0       0.0  0.0
59399  140345        4.0       0.0  0.0
60197  138694        4.0       0.0  0.

C:\Users\phamh\AppData\Local\Temp\ipykernel_22736\111309359.py:5: DtypeWarning: Columns (8,9,10,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_health = pd.read_csv("cleaned_data/Brand_Health_cleaned.csv")


In [21]:
import pandas as pd
import numpy as np

# Load dữ liệu
df_sa = pd.read_csv("cleaned_data/SA_var_cleaned.csv")
df_health = pd.read_csv("cleaned_data/Brand_Health_cleaned.csv")
df_seg = pd.read_csv("cleaned_data/Segmentation_cleaned.csv")

# Chuẩn hóa tên cột về chữ thường để tránh lỗi
df_sa.columns = df_sa.columns.str.lower()
df_health.columns = df_health.columns.str.lower()
df_seg.columns = df_seg.columns.str.lower()

# Chọn ID có Year = 2017
id_2017 = df_sa[df_sa['year'] == 2017]['id'].unique()

# Lọc dữ liệu liên quan
df_health_2017 = df_health[df_health['id'].isin(id_2017)].copy()
df_seg_2017 = df_seg[df_seg['id'].isin(id_2017)].copy()

# Đổi tên để merge và so sánh
df_health_2017 = df_health_2017.rename(columns={
    'ppa': 'health_ppa',
    'spending': 'health_spending',
    'fre_visit': 'health_visit'
})

df_seg_2017 = df_seg_2017.rename(columns={
    'ppa': 'seg_ppa',
    'spending': 'seg_spending',
    'visit': 'seg_visit'
})

# Merge theo ID và Brand
merged = pd.merge(df_health_2017, df_seg_2017, on=['id', 'brand'], how='inner')

# Ép kiểu số
for col in ['health_ppa', 'health_spending', 'health_visit', 'seg_ppa', 'seg_spending', 'seg_visit']:
    merged[col] = pd.to_numeric(merged[col], errors='coerce')

# Tính lại PPA
merged['seg_ppa_calc'] = merged['seg_spending'] / merged['seg_visit']

# Tính lệch tuyệt đối và lệch tương đối
merged['abs_diff_ppa'] = np.abs(merged['health_ppa'] - merged['seg_ppa_calc'])

condition = (
    (merged['abs_diff_ppa'] > 1) |
    ((merged['abs_diff_ppa'] / merged['health_ppa'].replace(0, np.nan)) > 0.2)
)

# Lọc inconsistency
inconsistencies = merged[condition]

print(f"Số dòng inconsistency A1: {len(inconsistencies)}")
print(inconsistencies[['id', 'brand', 'health_ppa', 'seg_ppa_calc', 'seg_ppa', 'abs_diff_ppa']].head())


Số dòng inconsistency A1: 0
Empty DataFrame
Columns: [id, brand, health_ppa, seg_ppa_calc, seg_ppa, abs_diff_ppa]
Index: []


C:\Users\phamh\AppData\Local\Temp\ipykernel_22736\3033574368.py:6: DtypeWarning: Columns (8,9,10,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_health = pd.read_csv("cleaned_data/Brand_Health_cleaned.csv")


In [22]:
df_health.columns = df_health.columns.str.lower()
df_sa.columns = df_sa.columns.str.lower()
df_seg.columns = df_seg.columns.str.lower()

# Bây giờ cột 'ID' trở thành 'id', 'Segmentation' thành 'segmentation'
df_merge_c1 = df_health[['id', 'segmentation']].merge(
    df_sa[['id']], on='id', how='inner'
).merge(
    df_seg[['id', 'segmentation']], on='id', how='left', suffixes=('_health', '_seg')
)

# So sánh segmentation giữa các bảng
df_c1_inconsistent = df_merge_c1[
    (df_merge_c1['segmentation_health'].notna()) &
    (df_merge_c1['segmentation_seg'].notna()) &
    (df_merge_c1['segmentation_health'] != df_merge_c1['segmentation_seg'])
]

print(f"C1 – Số dòng segmentation không khớp: {len(df_c1_inconsistent)}")
display(df_c1_inconsistent[['id', 'segmentation_health', 'segmentation_seg']])


C1 – Số dòng segmentation không khớp: 0


,id,segmentation_health,segmentation_seg


In [24]:
import pandas as pd
import numpy as np

# Load tất cả datasets
df_health = pd.read_csv("cleaned_data/Brand_Health_cleaned.csv", dtype=str, low_memory=False)
df_sa = pd.read_csv("cleaned_data/SA_var_cleaned.csv", dtype=str)
df_seg = pd.read_csv("cleaned_data/Segmentation_cleaned.csv", dtype=str)
df_dow = pd.read_csv("cleaned_data/Dayofweek_cleaned.csv", dtype=str)
df_dp = pd.read_csv("cleaned_data/Daypart_cleaned.csv", dtype=str)
df_comp = pd.read_csv("cleaned_data/Competitor_cleaned.csv", dtype=str)

# Chuẩn hóa tên cột thành chữ thường để tránh lỗi về KeyError
df_health.columns = df_health.columns.str.lower()
df_sa.columns = df_sa.columns.str.lower()
df_seg.columns = df_seg.columns.str.lower()
df_dow.columns = df_dow.columns.str.lower()
df_dp.columns = df_dp.columns.str.lower()
df_comp.columns = df_comp.columns.str.lower()

# Chuyển các cột cần thiết sang kiểu số
df_health['fre_visit'] = pd.to_numeric(df_health['fre_visit'], errors='coerce')
df_health['spending'] = pd.to_numeric(df_health['spending'], errors='coerce')
df_health['spending_use'] = pd.to_numeric(df_health['spending_use'], errors='coerce')
df_dow['visit#dayofweek'] = pd.to_numeric(df_dow['visit#dayofweek'], errors='coerce')
df_dp['visit#daypart'] = pd.to_numeric(df_dp['visit#daypart'], errors='coerce')
df_sa['group_size'] = pd.to_numeric(df_sa['group_size'], errors='coerce')

# =============================
# ✅ C1. Kiểm tra sự không khớp của segmentation giữa các bảng
# =============================
# Ghép df_health (chỉ lấy id, segmentation) với df_sa (chỉ lấy id) rồi merge với df_seg (id, segmentation)
df_merge_c1 = df_health[['id', 'segmentation']].merge(
    df_sa[['id']], on='id', how='inner'
).merge(
    df_seg[['id', 'segmentation']], on='id', how='left', suffixes=('_health', '_seg')
)

# Lọc các dòng mà segmentation khác nhau (với cả 2 bảng đều khác NaN)
df_c1_inconsistent = df_merge_c1[
    (df_merge_c1['segmentation_health'].notna()) &
    (df_merge_c1['segmentation_seg'].notna()) &
    (df_merge_c1['segmentation_health'] != df_merge_c1['segmentation_seg'])
]

print(f"C1 – Số dòng segmentation không khớp: {len(df_c1_inconsistent)}")
display(df_c1_inconsistent[['id', 'segmentation_health', 'segmentation_seg']])

# =============================
# ✅ C4. Kiểm tra group_size phải >= 1
# =============================
df_c4_invalid = df_sa[(df_sa['group_size'] < 1) | (df_sa['group_size'].isna())]
print(f"C4 – Số dòng có group_size < 1 hoặc thiếu: {len(df_c4_invalid)}")
display(df_c4_invalid[['id', 'group_size']])

# =============================
# ✅ C5. Kiểm tra spending_use ≤ spending
# =============================
df_c5_invalid = df_health[
    (df_health['spending'].notna()) &
    (df_health['spending_use'].notna()) &
    (df_health['spending_use'] > df_health['spending'])
]
print(f"C5 – Số dòng spending_use > spending: {len(df_c5_invalid)}")
display(df_c5_invalid[['id', 'spending', 'spending_use']])

# =============================
# ✅ C3. Kiểm tra brand phải nằm trong danh sách brand hợp lệ (danh sách lấy từ Competitor)
# =============================
# Chuẩn hóa cột brand cho cả df_health và df_comp
df_health['brand'] = df_health['brand'].str.lower().str.strip()
df_comp['brand'] = df_comp['brand'].str.lower().str.strip()

# Lấy danh sách các brand hợp lệ
valid_brands = df_comp['brand'].dropna().unique()

# Lọc các dòng trong Brand_Health có brand không thuộc danh sách hợp lệ
df_c3_invalid = df_health[~df_health['brand'].isin(valid_brands)]
print(f"C3 – Số dòng có brand không hợp lệ: {len(df_c3_invalid)}")
display(df_c3_invalid[['id', 'brand']].drop_duplicates())
print("Các brand không hợp lệ trong Brand_Health:")
print(df_c3_invalid['brand'].value_counts())


C1 – Số dòng segmentation không khớp: 0


,id,segmentation_health,segmentation_seg


C4 – Số dòng có group_size < 1 hoặc thiếu: 0


,id,group_size


C5 – Số dòng spending_use > spending: 0


,id,spending,spending_use


C3 – Số dòng có brand không hợp lệ: 42050


,id,brand
363,349551,milano
364,349553,milano
365,349957,milano
366,350148,milano
367,351103,milano
...,...,...
74413,456854,other
74414,456857,other
74415,458063,other
74416,458098,other


Các brand không hợp lệ trong Brand_Health:
brand
street                    10669
other                      9733
milano                     4699
independent cafe           2717
aha cafe                   1852
urban station              1667
passio                     1627
thức coffee                1061
viva star                  1029
mê trang                    836
coffee bean & tea leaf      824
long cafe                   710
gong cha                    594
mộc miên                    585
đen đá                      520
effoc                       473
maxx coffee                 397
saigon café                 363
bonpas                      316
runam cafe                  316
nia cafe                    263
the coffee factory          256
koi cafe                    237
the cups coffee             133
cheese coffee               123
laha coffee                  50
Name: count, dtype: int64
